<h1 align = "center">Επεξεργασία Φωνής και Φυσικής Γλώσσας</h1> 
<h2 align = "center">1η Προπαρασκευαστική Εργασία</h2> 
<h3 align = "center"> Θεoδωρόπουλος Νικήτας -03115185</h3>
<h3 align = "center"> Καλλιώρα Δωροθέα - 03115176</h3>




<h3>Σκοπός:</h3>
Σκοπός της εργαστηριακής άσκησης είναι η δημιουργία ενός απλού ορθογράφου με χρήση μηχανών πεπερασμένων καταστάσεων (fst) με τη βοήθεια της βιβλιοθήκης openfst (v1.6.1). Θα χρησιμοποιήσουμε corpus απο δημόσια διαθέσιμα βιβλία για να δημιουργήσουμε ενα γλωσσικό μοντέλο σε μορφή λεξικού. Τέλος θα γίνει εισαγωγή σε αναπαραστάσεις word2vec. 

<h1> TODO </h1> Dori

***Βήμα 1:***
(α)
Κατασκευάζουμε το corpus με σύμπτηξη plain text βιβλίων, δημόσια διαθέσημα στο Project Gutenberg. Θα χρησιμοποιήσουμε corpus 2 βιβλίων The War of the Worlds by H. G. Wells και Grimms' Fairy Tales by Jacob Grimm and Wilhelm Grimm. 

(β)
Η τεχνικής της συνένωσης πολλών βιβλίων, εκτός απο το προφανές πλεονέκτημα του μεγαλύτερου train corpus, αυξάνει την αναγνωριστική ισχύ του μοντέλου αφού μπορεί να αντλήσει λεξιλόγιο απο διαφορετικές πηγές. Ακόμα μειώνει το bias, αφού ένας συγραφέας μπορεί να χρησιμοποιεί συγκεκριμένες λέξεις λόγω προτίμησης, εποχής συγγραφής και άλλων παραγόντων. 

![./img/book36.cover.jpg](./img/book36.jpg)
![./img/book36.cover.jpg](./img/book2591.jpg)

In [66]:
#!wget -P ./data/ http://www.gutenberg.org/files/36/36-0.txt
#!wget -P ./data/ http://www.gutenberg.org/files/2591/2591-0.txt

***Βήμα 2***  Niki
ΔΙαβαζουμε αρχειο κτλπ...


In [5]:
# Step 2

def identity_preprocess(str):
  return str

def readfile(path,preprocess=identity_preprocess):
  processed_txt=[]
  f = open(path, "r")
  for line in f:
    processed_txt = processed_txt + preprocess(line)
  return processed_txt 

def tokenize(s):
    s = s.strip().lower()
    s = ''.join(c if c.isalpha() else ' ' for c in s)
    # We replace all non-alpha characters with ' '
    s = s.replace('\n',' ')
    s = s.split()
    return s



In [81]:
#Step 2 (d)

#!pip install --user -U nltk
import nltk
nltk.download('punkt')

sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
sentence = """At eight o'clock on Thursday morning....Arthur didn't feel very good.
              He quickly rushed to the Doctor!"""
tokenizers = [tokenize,nltk.word_tokenize,sent_detector.tokenize]
names = ["custom","nltk punkt","nltk sentece detector"]
for tokenizer,name in zip(tokenizers,names):
    print(name,":\n",tokenizer(sentence),'\n')


custom :
 ['at', 'eight', 'o', 'clock', 'on', 'thursday', 'morning', 'arthur', 'didn', 't', 'feel', 'very', 'good', 'he', 'quickly', 'rushed', 'to', 'the', 'doctor'] 

nltk punkt :
 ['At', 'eight', "o'clock", 'on', 'Thursday', 'morning', '...', '.Arthur', 'did', "n't", 'feel', 'very', 'good', '.', 'He', 'quickly', 'rushed', 'to', 'the', 'Doctor', '!'] 

nltk sentece detector :
 ["At eight o'clock on Thursday morning....Arthur didn't feel very good.", 'He quickly rushed to the Doctor!'] 



[nltk_data] Downloading package punkt to
[nltk_data]     /home/dorotheakal/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


***Βήμα 2*** dori

(δ) Συμπεράσματα και συγκρίσεις:

Sentence Tokenizer

This tokenizer divides a text into a list of sentences by using an unsupervised algorithm to build a model for abbreviation words, collocations, and words that start sentences. It must be trained on a large collection of plaintext in the target language before it can be used.

The NLTK data package includes a pre-trained Punkt tokenizer for English.

Word punkt
Tokenize a string to split off punctuation other than periods


Tokenize() //diko mas
na to grapsoume



In [51]:
# Step 3
import random
random.seed()

text_1 = readfile('./data/36-0.txt',tokenize)
text_2 = readfile('./data/2591-0.txt',tokenize)
text = text_1 + text_2

print(len(text),"words overall")
word_corpus = list(set(text))
print(len(word_corpus),"unique words in corpus")

# Convert list of words to list of chars, get unique chars with set
alphabet = list(set([c for word in word_corpus for c in word]))
print(len(alphabet),"symbols in alphabet:")
print(alphabet)



169392 words overall
3540
9288 unique words in corpus
28 symbols in alphabet:
['y', 'w', 'l', 'ç', 'm', 'b', 'e', 'k', 'v', 't', 'r', 'o', 'x', 'd', 'æ', 'i', 'q', 'u', 'g', 'a', 'f', 'h', 'z', 'c', 's', 'n', 'j', 'p']


Με βάση το αλφάβητο που προέκυψε απο το word corpus δημιουργούμε το αρχείο συμβόλων εισόδου και εξόδου για τα fst.

In [115]:
# Step 4

def create_syms(alphabet):
    f = open("./chars.syms","w+")
    f.write(f'<epsilon>     0\n')  #for <epsilon> index 0
    for i in range(len(alphabet)):
        f.write(f'{alphabet[i]}     {i+50}\n')  #for other characters index i+50
        
create_syms(alphabet)
    

***Βήμα 5*** niki

(α)Ως μετρική για την απόσταση 2 λέξεων θα χρησιμοποιήσουμε την απόσταση Levenshtein (ή edit distance). Η απόσταση μπορέι να υπολογιστεί αναδρομικά με χρήση dynamic programming (DP). Ανάμεσα σε δύο λέξεις μπορούν να γίνουν τρείς τύποι μετατροπών: εισαγωγή ενός χαρακτήρα ($ \epsilon \rightarrow a$), μετατροπή ενός χαρακτήρα ($ a \rightarrow b$) και διαγραφή ενός χαρακτήρα ($a \rightarrow \epsilon$). Θεωρούμε αρχικά ίδιο κόστος για όλες τις μετατροπές.

Δημιουργούμε κατάλληλο fst με μία κατάσταση, αντιστοιχίζοντας κάθε χαρακτήρα σε κάθε άλλον και στο $\epsilon$, και το $\epsilon$ σε κάθε χαρακτήρα με βάρος 1. Αντιστοιχίζουμε ακόμα τον χαρακτήρα στον εαυτό του με βάρος 0. Αν πάρουμε το shortest path τότε η έξοδος θα είναι λέξη εισόδου, αφού το ελάχιστο βάρος προκύπτει αν δεν γίνει καμία μετατροπή. 

(β)Στην υλοποίηση που έγινε για το ερώτημα 5 έχουμε υποθέσει ότι όλα τα edits έχουν ίσο βάρος. Αυτή ειναι μια αφελής υπόθεση αφού κάθε είδος λάθους (παράλλειψη, επανάληψη ή λάθος τοποθέτηση γράμματος)  έχει την ίδια πιθανότητα εμφάνισης. Ιδανικά θα θέλαμε να γνωρίζουμε την κατανομή του λάθους, την οποία μπορούμε να εκτιμήσουμε απο ένα σύνολο δεδομένων. Για ενα ζεύγος (σωστή λέξη, λανθασμένη λέξη) μπορούμε να υπολογίσουμε τη διόρθωση που απαιτείται και η πιθανότητα θα προκύψει $ P[ a\rightarrow b] = \frac{|error = b \rightarrow a|}{|errors|}, \forall a,b \in \{A+\epsilon\} $, όπου $A$ το αλφάβητο. Αυτές εινα οι a priori πιθανότητες λαθών με βάση το training data. Αυτές μπορούν να χρησιμοποιηθούν για να υπολιστούν τα βάρη στο μοντέλο αναγνώρισης μας με fst, ώστε να είναι αντιστρόφως ανάλογα της πιθανότητας. Μια καλή συνάρτηση αντιστοίχησης που διατηρεί τις αναλογίες είναι η $\frac{1}{x}$.



In [116]:
#Step 5

# arc format: src dest ilabel olabel [weight]
# final state format: state [weight]
# lines may occur in any order except initial state must be first line
# unspecified weights default to 0.0 (for the library-default Weight type) 

f = open('lev.fst',"w+")

def format_arc(src,dst,src_sym,dst_sym,w,f,acceptor=False):
    if not acceptor:
        f.write("{} {} {} {} {} \n".format(src,dst,src_sym,dst_sym,w))
    else:
        f.write("{} {} {} \n".format(src,dst,src_sym))
letters =  alphabet

for i in range(0, len(letters)):
    format_arc(src=0, dst=0, src_sym="<epsilon>", dst_sym=letters[i], w=1,f=f)
    format_arc(src=0, dst=0, src_sym=letters[i], dst_sym=letters[i], w=0,f=f)
    format_arc(src=0, dst=0, src_sym=letters[i], dst_sym="<epsilon>", w=1,f=f)
    for j in range(0, len(letters)):
        if(j!=i):
            format_arc(src=0, dst=0, src_sym=letters[i], dst_sym=letters[j], w=1,f=f)   
f.write('0\n')
f.close()
    

In [117]:
!fstcompile --isymbols=chars.syms --osymbols=chars.syms  lev.fst lev.bin.fst
#!fstdraw --isymbols=chars.syms --osymbols=chars.syms -portrait lev.bin.fst | dot -Tjpg >lev.jpg

Ένας αποδοχέας είναι ένα fst όπου κάθε μετάβαση έχει ενα label (και προεραιτικά βάρος). Στην βιβλιοθήκη fst μπορέι να θεωρηθεί ως μετατροπέας με ίδιο input και output label. Κατασκευάζουμε εναν αποδοχέα που αποδέχεται κάθε λέξη του λεξικού. dori

In [136]:
#Step 6

def create_acceptor(words,file):
    f = open(file,"w+")
    s = 0
    final_states = []
    for word in words:
        format_arc(0,s+1,word[0],word[0],0,f,acceptor=True)
        s += 1
        for letter in word[1:]:
            format_arc(s,s+1,letter, letter, 0,f,acceptor=True)
            s += 1
        final_states.append(s)

    for state in final_states:
        f.write(f'{state}\n')
    f.close()    
create_acceptor(word_corpus,"acceptor.fst")

In [137]:
!fstcompile --isymbols=chars.syms --osymbols=chars.syms --acceptor acceptor.fst acceptor.bin.fst
#!fstdraw --isymbols=chars.syms --osymbols=chars.syms -portrait rosebud.bin1.fst | dot -Tjpg >acceptor.jpg

#TODO: Τι κάνουν οι απο κατω συναρτήσεις απο την βιβλιοθήκη..... niki

In [138]:
#Step 6b

!fstrmepsilon acceptor.bin.fst acceptor.bin.fst
#!fstdraw --isymbols=chars.syms --osymbols=chars.syms -portrait acceptor.bin.fst | dot -Tjpg >dfa_rmepsilon.jpg

!fstdeterminize acceptor.bin.fst acceptor.bin.fst
#!fstdraw --isymbols=chars.syms --osymbols=chars.syms -portrait acceptor.bin.fst | dot -Tjpg >dfa_determinize.jpg

!fstminimize acceptor.bin.fst acceptor.bin.fst
#!fstdraw --isymbols=chars.syms --osymbols=chars.syms -portrait acceptor.bin.fst | dot -Tjpg >dfa_minimize.jpg


In [139]:
#Step 7
#mini.fst - lev.fst

!fstarcsort --sort_type=olabel lev.bin.fst lev.bin.fst
!fstarcsort --sort_type=ilabel acceptor.bin.fst acceptor.bin.fst

In [140]:
!fstcompose  lev.bin.fst acceptor.bin.fst spell_checker.bin.fst
#!fstdraw --isymbols=chars.syms --osymbols=chars.syms -portrait spell_checker.bin.fst | dot -Tjpg >com.jpg

In [158]:
create_acceptor(['cit'],"in.fst")

In [159]:
!fstcompile  --isymbols=chars.syms --osymbols=chars.syms  --acceptor in.fst in.bin.fst

In [160]:
!fstarcsort --sort_type=ilabel spell_checker.bin.fst spell_checker.bin.fst 
!fstarcsort --sort_type=olabel in.bin.fst in.bin.fst 

In [161]:
!fstcompose in.bin.fst spell_checker.bin.fst |fstshortestpath --nshortest=1 \
| fstrmepsilon |  fsttopsort |fstprint -isymbols=chars.syms  -osymbols=chars.syms\
| cut -f4 | grep -v "<epsilon>" |head -n -1 | tr -d '\n'


cut

In [162]:
!fstcompose in.bin.fst spell_checker.bin.fst |fstshortestpath --nshortest=1   \
| fstrmepsilon |  fsttopsort |fstprint -isymbols=chars.syms  -osymbols=chars.syms

0	1	c	c
1	2	i	u	1
2	3	t	t
3


In [163]:
#!wget https://raw.githubusercontent.com/georgepar/python-lab/master/spell_checker_test_set -P ./data/

In [175]:
# Step 8:

file = open('./data/spell_checker_test_set',"r")
y_test = []
X_test = []
for line in file:
    [y,X] = line.split(':')
    X = X.split()
    for word in X:
        X_test.append(word)
        y_test.append(y)

print(y_test[0:4])
print(X_test[0:4])
print(len(y_test))
print(len(X_test))
idxs = random.sample(range(0, len(y_test)), 20)
print(idxs)


['contented', 'contented', 'contented', 'contented']
['contenpted', 'contende', 'contended', 'contentid']
270
270
[133, 34, 43, 23, 138, 127, 260, 247, 29, 81, 221, 251, 188, 214, 130, 76, 1, 192, 110, 217]


In [176]:
def predict(Y,X):
    correct_pred=0
    for y,x in zip(Y,X):
        create_acceptor([x],"input.fst")

        !fstcompile  --isymbols=chars.syms --osymbols=chars.syms  --acceptor input.fst input.bin.fst
        !fstarcsort --sort_type=ilabel spell_checker.bin.fst spell_checker.bin.fst 
        !fstarcsort --sort_type=olabel input.bin.fst input.bin.fst 
        prediction = !fstcompose input.bin.fst spell_checker.bin.fst |fstshortestpath --nshortest=1 \
        | fstrmepsilon |  fsttopsort |fstprint -isymbols=chars.syms  -osymbols=chars.syms\
        | cut -f4 | grep -v "<epsilon>" |head -n -1 | tr -d '\n' 
        print("Wrong = ",x,"-- Correct = ",y,"-- Prediction = ",prediction[0])
        if y == prediction[0]:
            correct_pred+=1
    print(f"Accuracy:{correct_pred/len(Y)}%")


In [177]:
X_rand = [X_test[i] for i in idxs]
Y_rand = [y_test[i] for i in idxs]
predict(Y_rand,X_rand)

Wrong =  protend -- Correct =  pretend -- Prediction =  pretend
Wrong =  unexpcted -- Correct =  unexpected -- Prediction =  expected
Wrong =  biscuts -- Correct =  biscuits -- Prediction =  biscuits
Wrong =  independant -- Correct =  independent -- Prediction =  independent
Wrong =  wote -- Correct =  wrote -- Prediction =  note
Wrong =  chaphter -- Correct =  chapter -- Prediction =  chapter
Wrong =  adres -- Correct =  address -- Prediction =  agree
Wrong =  oppisit -- Correct =  opposite -- Prediction =  opposite
Wrong =  poety -- Correct =  poetry -- Prediction =  piety
Wrong =  totaly -- Correct =  totally -- Prediction =  coaly
Wrong =  failes -- Correct =  fails -- Prediction =  files
Wrong =  oppossitte -- Correct =  opposite -- Prediction =  opposite
Wrong =  planed -- Correct =  planned -- Prediction =  plane
Wrong =  embaras -- Correct =  embarrass -- Prediction =  mars
Wrong =  vairious -- Correct =  various -- Prediction =  various
Wrong =  arnt -- Correct =  aunt -- Pred

In [29]:
#Step 9
def tokenized_list(path,preprocess=identity_preprocess):
  list_of_sentences=[]
  f = open(path, "r")
  for line in f:
    l = preprocess(line)
    if l:
        list_of_sentences.append(l)
  return list_of_sentences


list1 = tokenized_list('./data/36-0.txt',tokenize)
list2 = tokenized_list('./data/2591-0.txt',tokenize)
final_list = list1 + list2 
print(len(final_list))


13945


In [22]:
# Initialize word2vec. Context is taken as the 2 previous and 2 next words
!pip install -U gensim
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec

Requirement already up-to-date: gensim in /home/dorotheakal/anaconda3/lib/python3.7/site-packages (3.8.1)


In [25]:
model = Word2Vec(final_list, window=5, size=100, workers=4)
model.train(final_list, total_examples=len(final_list), epochs=1000)

(108772021, 169392000)

In [62]:
import random

# get ordered vocabulary list
voc = model.wv.index2word
print(len(voc))
# get vector size
dim = model.vector_size
print(dim)
# get most similar words
sim = model.wv.most_similar('cut')
print(sim)

# Convert to numpy 2d array (n_vocab x vector_size)
def to_embeddings_Matrix(model):  
    embedding_matrix = np.zeros((len(model.wv.vocab), model.vector_size))
    word2idx = {}
    for i in range(len(model.wv.vocab)):
        embedding_matrix[i] = model.wv[model.wv.index2word[i]]
        word2idx[model.wv.index2word[i]] = i
    return embedding_matrix, model.wv.index2word, word2idx

# word2idx : "word" -> index
# embedding_matrix : index -> embedding 
# model.wv.index2word : index -> "word"

idxs = random.sample(range(0, len(voc)), 10)
rand_words = [voc[i] for i in idxs]
for word in rand_words:
    print(f'Most similar words to "{word}":')
    for word,sim in model.wv.most_similar(word):
        print(f'     "{word}" -- sim: {sim}')


2811
100
[('throwing', 0.3889895975589752), ('tore', 0.3638570308685303), ('threw', 0.35819464921951294), ('took', 0.34865516424179077), ('pay', 0.34183257818222046), ('set', 0.3053753972053528), ('gave', 0.3050578534603119), ('take', 0.3020142614841461), ('broke', 0.30198562145233154), ('bring', 0.2947586178779602)]
Most similar words to "greenish":
     "vapour" -- sim: 0.40728604793548584
     "violent" -- sim: 0.3979704976081848
     "puffs" -- sim: 0.39006245136260986
     "foam" -- sim: 0.38760900497436523
     "whirling" -- sim: 0.3635241389274597
     "haze" -- sim: 0.3582707345485687
     "flag" -- sim: 0.35365885496139526
     "gigantic" -- sim: 0.3512701988220215
     "green" -- sim: 0.3445907235145569
     "tumult" -- sim: 0.341741681098938
Most similar words to "everybody":
     "rubbish" -- sim: 0.3829894959926605
     "health" -- sim: 0.3715789318084717
     "defect" -- sim: 0.35363760590553284
     "provide" -- sim: 0.3510478138923645
     "poured" -- sim: 0.33493357896

In [37]:
import numpy as np


#to_embeddings_Matrix(model)

3